## Exploring machine learning models to predict startups probability of reaching TRL 6-8

#### Data source: NetZero Insights (https://netzeroinsights.com/)
#### Filtering criteria: 
#### 1. Founded year from 2021-2024
#### 2. Stage: Venture Capital stage
#### 3. Startup sectors: Energy Hardware Solutions; GHG Capture, Removal, and Storage Hardware Solutions

-----
### Preparation: Load relevant Python packages


In [1]:
import pandas as pd
import numpy as np
import sklearn as sklearn

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt

### Load data from Excel

In [2]:
NZI = 'Net0i_organisations_2024-10-01.xlsx'
startups = pd.read_excel(NZI)

### Check dataframe before cleaning up data

In [3]:
# Initial dataframe check
startups.head()
startups.info()
startups.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Net0i Organisation ID      731 non-null    int64  
 1   Name                       731 non-null    object 
 2   Website                    721 non-null    object 
 3   Domain                     721 non-null    object 
 4   Pitch line                 730 non-null    object 
 5   Founded date               731 non-null    int64  
 6   Continent                  693 non-null    object 
 7   HQ country                 693 non-null    object 
 8   HQ country code            692 non-null    object 
 9   City                       645 non-null    object 
 10  Email                      582 non-null    object 
 11  Phone                      321 non-null    object 
 12  Employees                  697 non-null    float64
 13  QoQ Employees Growth       661 non-null    float64

Index(['Net0i Organisation ID', 'Name', 'Website', 'Domain', 'Pitch line',
       'Founded date', 'Continent', 'HQ country', 'HQ country code', 'City',
       'Email', 'Phone', 'Employees', 'QoQ Employees Growth',
       'QoQ Corresponding quarter', 'YoY Employees Growth',
       'YoY Corresponding quarter', 'Stage', 'Total funding USD',
       'Announced last deal date', 'Announced last deal type',
       'Last deal amount USD', 'Total deal count', 'Tags', 'TRL', 'SDGs',
       'LinkedIn URL', 'Twitter URL', 'Facebook URL', 'Logo URL',
       'Net0i profile URL', 'Net0 score', 'Net0 score bucket',
       'Last viewed date', 'Notes'],
      dtype='object')

### Data preprocessing
#### 1. Drop irrelevant columns and rows with NaN values for important numerical parameters

In [4]:
# Drop the columns with irrelevant data, namely:
# Net0i Organisation ID (0), Name (1), Domain (3), Pitch line (4), HQ Country code (8), City (9), Email (10), Phone (11), Tags (23), SDGs (25), Net0i profile URL (30), Net0 score (31), Net0 score bucket (32), Last viewed date (33), Notes (34)
columns_to_drop_for_ML = [0, 1, 3, 4, 8, 9, 10, 11, 23, 25, 30, 31, 32, 33, 34]
startups = startups.drop(startups.columns[columns_to_drop_for_ML], axis=1)

# Drop rows with NaN values in critical numerical data
startups = startups.dropna(subset=['Employees'])
startups = startups.dropna(subset=['Total deal count'])

#### 2. Converting columns with string values to boolean

In [5]:
# Replace the column 'Website', 'LinkedIn URL', 'Twitter URL', 'Facebook URL', 'Logo URL' with boolean:
# eg. startup has Website = 1, has no Website = 0
startups['Website'] = startups['Website'].apply(lambda x: 1 if pd.notna(x) and x.strip() != '' else 0)
startups['LinkedIn URL'] = startups['LinkedIn URL'].apply(lambda x: 1 if pd.notna(x) and x.strip() != '' else 0)
startups['Twitter URL'] = startups['Twitter URL'].apply(lambda x: 1 if pd.notna(x) and x.strip() != '' else 0)
startups['Facebook URL'] = startups['Facebook URL'].apply(lambda x: 1 if pd.notna(x) and x.strip() != '' else 0)
startups['Logo URL'] = startups['Logo URL'].apply(lambda x: 1 if pd.notna(x) and x.strip() != '' else 0)

#### 3. Ensure that integers are formatted correctly

In [6]:
# Make sure that 'Founded date', 'Employees', 'Round count' are all integers
startups['Founded date'] = startups['Founded date'].apply(lambda x: int(x))
startups['Employees'] = startups['Employees'].apply(lambda x: int(x))
startups['Total deal count'] = startups['Total deal count'].apply(lambda x: int(x))

# Interim dataframe check
startups.head()

,Website,Founded date,Continent,HQ country,Employees,QoQ Employees Growth,QoQ Corresponding quarter,YoY Employees Growth,YoY Corresponding quarter,Stage,Total funding USD,Announced last deal date,Announced last deal type,Last deal amount USD,Total deal count,TRL,LinkedIn URL,Twitter URL,Facebook URL,Logo URL
0,1,2021,North America,United States,121,-0.0397,Q1 2024 vs Q4 2023,0.0522,Q1 2024 vs Q1 2023,Growth,73000000,2023-11-15,Debt,20000000.0,4,NaN,1,1,1,1
1,1,2021,Europe,United Kingdom,6,0.0000,Q2 2024 vs Q1 2024,0.5000,Q2 2024 vs Q2 2023,Ideation,53629,2024-04-04,Accelerator/incubator,53629.0,2,1-5,1,0,0,1
3,1,2021,North America,Canada,25,0.0000,Q3 2024 vs Q1 2024,0.1905,Q3 2024 vs Q3 2023,Growth,16087572,2024-07-03,Grant,2187572.0,7,6-8,1,0,0,1
4,1,2021,North America,United States,6,0.0000,Q2 2024 vs Q1 2024,0.5000,Q2 2024 vs Q2 2023,Ideation,85000,2024-03-14,Grant,NaN,4,9,1,1,1,1
5,1,2022,Europe,United Kingdom,10,0.0000,Q1 2024 vs Q4 2023,0.1111,Q1 2024 vs Q1 2023,Ideation,0,2023-01-09,Accelerator/incubator,NaN,3,6-8,1,0,0,1


#### 4. Look for missing values and handle missing values

##### There should only be missing values for TRL since that is the target variable for machine learning models

In [7]:
# Find missing values
if startups.isnull().sum().sum() == 0:
    print("There are no missing values now!")
else:
    null = pd.DataFrame(startups.isnull().sum(), columns=["Null Values"])
    null["% Missing Values"] = (startups.isna().sum() / len(startups) * 100)
    null = null[null["% Missing Values"] > 0]
    print("There are still missing values:")
    display(null.style.background_gradient(cmap='viridis', low=0.2, high=0.1))

There are still missing values:


,Null Values,% Missing Values
Continent,36,5.164993
HQ country,36,5.164993
QoQ Employees Growth,36,5.164993
QoQ Corresponding quarter,36,5.164993
YoY Employees Growth,36,5.164993
YoY Corresponding quarter,36,5.164993
Announced last deal date,52,7.460545
Announced last deal type,11,1.578192
Last deal amount USD,409,58.680057
TRL,530,76.040172


In [8]:
# Handle missing values for 'Last deal amount USD'
startups['Last deal amount USD'] = startups['Last deal amount USD'].fillna(startups['Total funding USD'] / startups['Total deal count'])

#### 5. Convert string data to datetime format for deal dates 
##### More details in line with code

In [9]:
# Convert 'Announced last deal date' to datetime format
startups['Announced last deal date'] = pd.to_datetime(startups['Announced last deal date'])

# Convert QoQ and YoY to datetime format
startups['QoQ Corresponding quarter'] = startups['QoQ Corresponding quarter'].astype(str)
startups['YoY Corresponding quarter'] = startups['YoY Corresponding quarter'].astype(str)

# Define a new column 'QoQ_employee change' as the time where there is a change in employee number: 
startups['QoQ_employee change'] = startups['QoQ Corresponding quarter'].apply(lambda x: x.split(' vs ')[0] if ' vs ' in x else np.nan)
startups['YoY_employee change'] = startups['YoY Corresponding quarter'].apply(lambda x: x.split(' vs ')[0] if ' vs ' in x else np.nan)

# Define a function to convert quarter info to datetime
def quarter_to_date(quarter_str):
    if pd.isna(quarter_str):
        return np.nan
    quarter, year = quarter_str.split()
    year = int(year)
    if quarter == 'Q1':
        return pd.Timestamp(year=year, month=1, day=1)
    elif quarter == 'Q2':
        return pd.Timestamp(year=year, month=4, day=1)
    elif quarter == 'Q3':
        return pd.Timestamp(year=year, month=7, day=1)
    elif quarter == 'Q4':
        return pd.Timestamp(year=year, month=10, day=1)

# Convert to datetime format using the above function
startups['QoQ_datetime'] = startups['QoQ_employee change'].apply(quarter_to_date)
startups['YoY_datetime'] = startups['YoY_employee change'].apply(quarter_to_date)

startups['QoQ_year'] = startups['QoQ_datetime'].apply(lambda time: time.year)
startups['QoQ_year - Founded date'] = startups['QoQ_year'] - startups['Founded date']

### Map TRL level to convert string to integer 
#### TRL = 1-5 mapped to 1: Research phase
#### TRL = 6-8 mapped to 2: Development phase
#### TRL = 9   mapped to 3: Deployment phase

In [10]:
## Map TRL level ##
# TRL = 1-5 mapped to 1: research phase
# TRL = 6-8 mapped to 2: development phase
# TRL = 9   mapped to 3: deployment phase
trl_mapping = {
    '1-5': 1,
    '6-8': 2,
    '9': 3
}

# Apply the mapping to the TRL column
startups['TRL mapping'] = startups['TRL'].map(trl_mapping)

### Sense check for numeric and categorical data

In [11]:
# Find numeric data
numeric=['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
startups_num =startups.select_dtypes(include=numeric)

print("\nNumeric parameters:")
startups_num.head(3)

# Find categorical data
startups_cat = startups.select_dtypes(include='object')

print("\nCategorical parameters:")
startups_cat.head(3)


Numeric parameters:

Categorical parameters:


,Continent,HQ country,QoQ Corresponding quarter,YoY Corresponding quarter,Stage,Announced last deal type,TRL,QoQ_employee change,YoY_employee change
0,North America,United States,Q1 2024 vs Q4 2023,Q1 2024 vs Q1 2023,Growth,Debt,NaN,Q1 2024,Q1 2024
1,Europe,United Kingdom,Q2 2024 vs Q1 2024,Q2 2024 vs Q2 2023,Ideation,Accelerator/incubator,1-5,Q2 2024,Q2 2024
3,North America,Canada,Q3 2024 vs Q1 2024,Q3 2024 vs Q3 2023,Growth,Grant,6-8,Q3 2024,Q3 2024


In [12]:
# Find missing values with an if loop
if startups.isnull().sum().sum() == 0:
    print("There are no missing values now!")
else:
    null = pd.DataFrame(startups.isnull().sum(), columns=["Null Values"])
    null["% Missing Values"] = (startups.isna().sum() / len(startups) * 100)
    null = null[null["% Missing Values"] > 0]
    print("There are still missing values:")
    display(null.style.background_gradient(cmap='viridis', low=0.2, high=0.1))

There are still missing values:


,Null Values,% Missing Values
Continent,36,5.164993
HQ country,36,5.164993
QoQ Employees Growth,36,5.164993
YoY Employees Growth,36,5.164993
Announced last deal date,52,7.460545
Announced last deal type,11,1.578192
TRL,530,76.040172
QoQ_employee change,36,5.164993
YoY_employee change,36,5.164993
QoQ_datetime,36,5.164993


### Use correlation heatmap to check data correlation
#### Correlation heatmap outputs a matrix of correlation values
The bigger the absolute number, the stronger the correlation. 
##### - Positive number means positively correlated. 
##### - Negative number means negatively correlated. 

In [13]:
# First drop the categorical data because correlation heatmap analyzes numeric data
corr_heatmap = startups.drop(['Website', 'Continent', 'HQ country', 'QoQ Corresponding quarter', 'YoY Corresponding quarter', 'Stage', 'Announced last deal type', 'TRL', 'LinkedIn URL', 'Twitter URL', 'Facebook URL', 'Logo URL', 'QoQ_employee change', 'YoY_employee change'], axis=1)

# Check data correlation with correlation heatmap
corr_heatmap.corr()

# Sort the correlation values in comparison with TRL mapping
corr_heatmap.corr()['TRL mapping'].sort_values(ascending=True)

Founded date              -0.178756
Total deal count           0.052527
QoQ Employees Growth       0.095453
YoY Employees Growth       0.133745
QoQ_year - Founded date    0.180180
Total funding USD          0.231495
Last deal amount USD       0.272486
Employees                  0.305063
TRL mapping                1.000000
QoQ_year                        NaN
Name: TRL mapping, dtype: float64

### Remove other missing values that does not affect model prediction 
##### See in line comments for detailed code operations

In [14]:
# Find the datetime columns to see if they are OK to be dropped
datetime_columns = startups.select_dtypes(include=['datetime64[ns]']).columns
print(datetime_columns)

Index(['Announced last deal date', 'QoQ_datetime', 'YoY_datetime'], dtype='object')


In [15]:
# Remove columns with datetime
startups = startups.select_dtypes(exclude=['datetime64[ns]'])

# List of columns to check for missing data
columns_to_check = [
    'Continent', 
    'HQ country', 
    'QoQ Employees Growth', 
    'YoY Employees Growth',
    'Announced last deal type'
]

# Drop rows with missing data in any of the specified columns
startups = startups.dropna(subset=columns_to_check)

# Define numeric and categorical columns
numeric_features = startups.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = startups.select_dtypes(include=['object']).columns.tolist()

# Define the preprocessing for numeric features (impute missing values)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

# Define the preprocessing for categorical features (impute missing values and one-hot encode)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine the preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [16]:
# Check for missing values again with the same if loop as above
if startups.isnull().sum().sum() == 0:
    print("There are no missing values now!")
else:
    null = pd.DataFrame(startups.isnull().sum(), columns=["Null Values"])
    null["% Missing Values"] = (startups.isna().sum() / len(startups) * 100)
    null = null[null["% Missing Values"] > 0]
    print("There are still missing values:")
    display(null.style.background_gradient(cmap='viridis', low=0.2, high=0.1))

There are still missing values:


,Null Values,% Missing Values
TRL,467,75.934959
TRL mapping,467,75.934959


## ----- End of basic data preprocessing ----- ##

### Define target columns and convert non-numeric columns to numeric

In [17]:
# Define the target columns as a standalone variable to streamline the code
target_columns = ['TRL mapping', 'TRL']
target_column = 'TRL mapping'

# Save the target_column as a standalone dataframe
target_column_data = startups[target_column]
# dropping both 'TRL mapping' and 'TRL' 
startups = startups.drop(columns=target_columns)

# Convert non-numeric columns to numeric, now excluding the target_columns
startups = pd.get_dummies(startups, drop_first=True)

# Add target_column_data back to the dataframe (see only target_column ('TRL mapping') is added back!!)
startups[target_column] = target_column_data

In [18]:
startups.head(3)

,Website,Founded date,Employees,QoQ Employees Growth,YoY Employees Growth,Total funding USD,Last deal amount USD,Total deal count,LinkedIn URL,Twitter URL,...,Announced last deal type_Series B,Announced last deal type_Series C,Announced last deal type_Spinoff/spinout,QoQ_employee change_Q2 2024,QoQ_employee change_Q3 2024,QoQ_employee change_Q4 2023,YoY_employee change_Q2 2024,YoY_employee change_Q3 2024,YoY_employee change_Q4 2023,TRL mapping
0,1,2021,121,-0.0397,0.0522,73000000,20000000.0,4,1,1,...,0,0,0,0,0,0,0,0,0,NaN
1,1,2021,6,0.0000,0.5000,53629,53629.0,2,1,0,...,0,0,0,1,0,0,1,0,0,1.0
3,1,2021,25,0.0000,0.1905,16087572,2187572.0,7,1,0,...,0,0,0,0,1,0,0,1,0,2.0


### Save the dataframe to two separate indices
#### train_data: dataframe with TRL information available
#### predict_data: dataframe with TRL information missing

In [19]:
# Save to prediction_data_idx if rows with 'TRL mapping' is NaN is True and to train_data_idx if False
train_data_idx = startups[target_column].notna()
predict_data_idx = startups[target_column].isna()

# Split data into training and inference data, with respect to the availability of TRL data
train_data = startups[train_data_idx]
predict_data = startups[predict_data_idx]
if target_column in predict_data.columns:
    predict_data.drop(labels=target_column, axis=1, inplace=True)

/var/folders/fg/dmf9kv792n12sqj119w84y200000gn/T/ipykernel_39182/1525523344.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data.drop(labels=target_column, axis=1, inplace=True)


In [20]:
# Make sure target data is of type int64
train_data = train_data.astype({target_column: 'int64'})

### The preprocessed data will then be saved as two separate sheets:
#### 1. Train_Data: to train the machine learning model 
#### 2. Predict_Data: to use the trained model to predict TRL 

In [21]:
# Save to one Excel file with two separate sheets
with pd.ExcelWriter('preprocessed_data.xlsx') as writer:
    train_data.to_excel(writer, sheet_name='Train_Data', index=False)
    predict_data.to_excel(writer, sheet_name='Predict_Data', index=False)

print("\nPreprocessed data is now saved under: preprocessed_data.xlsx")

predict_data.head()


Preprocessed data is now saved under: preprocessed_data.xlsx


,Website,Founded date,Employees,QoQ Employees Growth,YoY Employees Growth,Total funding USD,Last deal amount USD,Total deal count,LinkedIn URL,Twitter URL,...,Announced last deal type_Series A,Announced last deal type_Series B,Announced last deal type_Series C,Announced last deal type_Spinoff/spinout,QoQ_employee change_Q2 2024,QoQ_employee change_Q3 2024,QoQ_employee change_Q4 2023,YoY_employee change_Q2 2024,YoY_employee change_Q3 2024,YoY_employee change_Q4 2023
0,1,2021,121,-0.0397,0.0522,73000000,20000000.0,4,1,1,...,0,0,0,0,0,0,0,0,0,0
19,1,2022,26,1.3636,5.5000,2300000,1300000.0,2,1,0,...,0,0,0,0,0,1,0,0,1,0
45,1,2021,6,0.5000,0.0000,0,0.0,1,1,1,...,0,0,0,0,0,1,0,0,1,0
54,1,2021,21,0.0000,0.4000,250000000,250000000.0,3,1,1,...,0,0,0,0,0,0,0,0,0,0
55,1,2022,10,0.2500,1.5000,6250000,5000000.0,5,1,1,...,0,0,0,0,0,0,0,0,0,0


### Define target variable and feature variables
#### Target variable (y): the value that the model is trying to predict
#### Feature variables (X): the input parameters for the model to make prediction on "y"

In [22]:
# Define target variable and feature variables
y = train_data[target_column]  # Target
X = train_data.drop(columns=[target_column])  # Features

### Train-Validation-Test split

In [23]:
# Split the data into training, validation, and test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, shuffle=True)

# Scale only for Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [24]:
# Save the standard scaler as .bin
import joblib
joblib.dump(scaler, 'std_scaler_Op1.bin', compress=True)

['std_scaler_Op1.bin']

-----

## Logistic Regression Model

In [25]:
## LOGISTIC REGRESSION MODEL ##

# Initialize and train the model with scaled data
LR_model = LogisticRegression(max_iter=1000)  # Increase max_iter if needed
LR_model.fit(X_train_scaled, y_train)

# Make predictions on validation set
y_val_pred = LR_model.predict(X_val_scaled)
val_accuracy = accuracy_score(y_val, y_val_pred)

# Evaluate the validation set
print("Validation Accuracy:", val_accuracy)
print("Classification Report on Validation Set:\n", classification_report(y_val, y_val_pred, zero_division=1))
print("Confusion Matrix on Validation Set:\n", confusion_matrix(y_val, y_val_pred))

# Make predictions on test set
y_test_pred = LR_model.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Evaluate the test set
print("Test Accuracy:", test_accuracy)
print("Classification Report on Test Set:\n", classification_report(y_test, y_test_pred, zero_division=1))
print("Confusion Matrix on Test Set:\n", confusion_matrix(y_test, y_test_pred))

Validation Accuracy: 0.4666666666666667
Classification Report on Validation Set:
               precision    recall  f1-score   support

           1       0.56      0.67      0.61        15
           2       0.33      0.29      0.31         7
           3       0.33      0.25      0.29         8

    accuracy                           0.47        30
   macro avg       0.41      0.40      0.40        30
weighted avg       0.44      0.47      0.45        30

Confusion Matrix on Validation Set:
 [[10  2  3]
 [ 4  2  1]
 [ 4  2  2]]
Test Accuracy: 0.5
Classification Report on Test Set:
               precision    recall  f1-score   support

           1       0.63      0.67      0.65        18
           2       0.12      0.17      0.14         6
           3       0.67      0.33      0.44         6

    accuracy                           0.50        30
   macro avg       0.47      0.39      0.41        30
weighted avg       0.54      0.50      0.51        30

Confusion Matrix on Test Se

### Save the trained model

In [26]:
# Save the model to a .pkl file
joblib_file = "logistic_regression_model_Op1.pkl"
joblib.dump(LR_model, joblib_file)

['logistic_regression_model_Op1.pkl']

#### Sense check with predict_data to make sure there were no data issues

In [27]:
predict_data

,Website,Founded date,Employees,QoQ Employees Growth,YoY Employees Growth,Total funding USD,Last deal amount USD,Total deal count,LinkedIn URL,Twitter URL,...,Announced last deal type_Series A,Announced last deal type_Series B,Announced last deal type_Series C,Announced last deal type_Spinoff/spinout,QoQ_employee change_Q2 2024,QoQ_employee change_Q3 2024,QoQ_employee change_Q4 2023,YoY_employee change_Q2 2024,YoY_employee change_Q3 2024,YoY_employee change_Q4 2023
0,1,2021,121,-0.0397,0.0522,73000000,20000000.0,4,1,1,...,0,0,0,0,0,0,0,0,0,0
19,1,2022,26,1.3636,5.5000,2300000,1300000.0,2,1,0,...,0,0,0,0,0,1,0,0,1,0
45,1,2021,6,0.5000,0.0000,0,0.0,1,1,1,...,0,0,0,0,0,1,0,0,1,0
54,1,2021,21,0.0000,0.4000,250000000,250000000.0,3,1,1,...,0,0,0,0,0,0,0,0,0,0
55,1,2022,10,0.2500,1.5000,6250000,5000000.0,5,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,1,2021,9,0.1250,0.8000,0,0.0,3,1,0,...,0,0,0,0,0,1,0,0,1,0
726,1,2022,1,0.0000,0.0000,0,0.0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
727,1,2021,7,0.1667,6.0000,0,0.0,2,1,0,...,0,0,0,0,1,0,0,1,0,0
728,1,2022,10,-0.0909,0.1111,1600000,320000.0,5,1,0,...,0,0,0,0,1,0,0,1,0,0


### Make predictions using the logistic regression model: 

#### Code will return an Excel spreadsheet with:
#### 1. the TRL prediction and 
#### 2. the probability (or confidence level) for each predictions

In [28]:
# Load the model
LR_model = joblib.load(joblib_file)

# load standard scaler
scaler = joblib.load('std_scaler_Op1.bin')

# Load the prediction Excel file
file_path = 'preprocessed_data.xlsx'
predict_data = pd.read_excel(file_path, sheet_name='Predict_Data')
predict_data_scaled = scaler.transform(predict_data)

# Assuming new_data is already preprocessed and ready for predictions
predictions = LR_model.predict(predict_data_scaled)
probabilities = LR_model.predict_proba(predict_data_scaled)

# Add predictions and probabilities to the existing DataFrame
# predict_data_scaled is a numpy ndarray and has no labels 
# Add prediction to unscaled, original predict_data DataFrame
predict_data['Predicted_Class'] = predictions
predict_data['Probability_TRL_1'] = probabilities[:, 0]
predict_data['Probability_TRL_2'] = probabilities[:, 1]
predict_data['Probability_TRL_3'] = probabilities[:, 2]

predict_data

,Website,Founded date,Employees,QoQ Employees Growth,YoY Employees Growth,Total funding USD,Last deal amount USD,Total deal count,LinkedIn URL,Twitter URL,...,QoQ_employee change_Q2 2024,QoQ_employee change_Q3 2024,QoQ_employee change_Q4 2023,YoY_employee change_Q2 2024,YoY_employee change_Q3 2024,YoY_employee change_Q4 2023,Predicted_Class,Probability_TRL_1,Probability_TRL_2,Probability_TRL_3
0,1,2021,121,-0.0397,0.0522,73000000,20000000.0,4,1,1,...,0,0,0,0,0,0,3,0.236799,0.344340,0.418861
1,1,2022,26,1.3636,5.5000,2300000,1300000.0,2,1,0,...,0,1,0,0,1,0,2,0.007564,0.983151,0.009285
2,1,2021,6,0.5000,0.0000,0,0.0,1,1,1,...,0,1,0,0,1,0,1,0.815215,0.173738,0.011047
3,1,2021,21,0.0000,0.4000,250000000,250000000.0,3,1,1,...,0,0,0,0,0,0,3,0.000002,0.000199,0.999799
4,1,2022,10,0.2500,1.5000,6250000,5000000.0,5,1,1,...,0,0,0,0,0,0,1,0.998031,0.000944,0.001025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,1,2021,9,0.1250,0.8000,0,0.0,3,1,0,...,0,1,0,0,1,0,2,0.382651,0.537843,0.079506
463,1,2022,1,0.0000,0.0000,0,0.0,1,1,1,...,0,0,0,0,0,0,1,0.928266,0.024108,0.047626
464,1,2021,7,0.1667,6.0000,0,0.0,2,1,0,...,1,0,0,1,0,0,3,0.000286,0.042096,0.957618
465,1,2022,10,-0.0909,0.1111,1600000,320000.0,5,1,0,...,1,0,0,1,0,0,1,0.886908,0.032591,0.080501


### Save the performed predictions to a new Excel file

In [29]:
# Save the updated DataFrame to an Excel file
predict_data.to_excel("Predictions with logistic regression model.xlsx", index=False)

-----

## Decision Tree Model

In [30]:
## DECISION TREE MODEL ##

DT_model = DecisionTreeClassifier(random_state=42)
DT_model.fit(X_train, y_train)

# Make predictions on validation set
y_val_pred = DT_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)

# Evaluate the validation set
print("Validation Accuracy:", val_accuracy)
print("Classification Report on Validation Set:\n", classification_report(y_val, y_val_pred, zero_division=1))
print("Confusion Matrix on Validation Set:\n", confusion_matrix(y_val, y_val_pred))

# Make predictions on test set
y_test_pred = LR_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Evaluate the test set
print("Test Accuracy:", test_accuracy)
print("Classification Report on Test Set:\n", classification_report(y_test, y_test_pred, zero_division=1))
print("Confusion Matrix on Test Set:\n", confusion_matrix(y_test, y_test_pred))

Validation Accuracy: 0.5
Classification Report on Validation Set:
               precision    recall  f1-score   support

           1       0.64      0.60      0.62        15
           2       0.45      0.71      0.56         7
           3       0.20      0.12      0.15         8

    accuracy                           0.50        30
   macro avg       0.43      0.48      0.44        30
weighted avg       0.48      0.50      0.48        30

Confusion Matrix on Validation Set:
 [[9 3 3]
 [1 5 1]
 [4 3 1]]
Test Accuracy: 0.4
Classification Report on Test Set:
               precision    recall  f1-score   support

           1       1.00      0.33      0.50        18
           2       1.00      0.00      0.00         6
           3       0.25      1.00      0.40         6

    accuracy                           0.40        30
   macro avg       0.75      0.44      0.30        30
weighted avg       0.85      0.40      0.38        30

Confusion Matrix on Test Set:
 [[ 6  0 12]
 [ 0  0 

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


-----

## Random Forest Model

In [31]:
## RANDOM FOREST MODEL ##

RF_model = RandomForestClassifier(n_estimators=100, random_state=42)
RF_model.fit(X_train, y_train)

# Make predictions on validation set
y_val_pred = RF_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)

# Evaluate the validation set
print("Validation Accuracy:", val_accuracy)
print("Classification Report on Validation Set:\n", classification_report(y_val, y_val_pred, zero_division=1))
print("Confusion Matrix on Validation Set:\n", confusion_matrix(y_val, y_val_pred))

# Make predictions on test set
y_test_pred = LR_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Evaluate the test set
print("Test Accuracy:", test_accuracy)
print("Classification Report on Test Set:\n", classification_report(y_test, y_test_pred, zero_division=1))
print("Confusion Matrix on Test Set:\n", confusion_matrix(y_test, y_test_pred))

Validation Accuracy: 0.43333333333333335
Classification Report on Validation Set:
               precision    recall  f1-score   support

           1       0.48      0.80      0.60        15
           2       0.33      0.14      0.20         7
           3       0.00      0.00      0.00         8

    accuracy                           0.43        30
   macro avg       0.27      0.31      0.27        30
weighted avg       0.32      0.43      0.35        30

Confusion Matrix on Validation Set:
 [[12  1  2]
 [ 6  1  0]
 [ 7  1  0]]
Test Accuracy: 0.4
Classification Report on Test Set:
               precision    recall  f1-score   support

           1       1.00      0.33      0.50        18
           2       1.00      0.00      0.00         6
           3       0.25      1.00      0.40         6

    accuracy                           0.40        30
   macro avg       0.75      0.44      0.30        30
weighted avg       0.85      0.40      0.38        30

Confusion Matrix on Test S

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


-----

## Gradient Boosting Model

In [32]:
## GRADIENT BOOSTING MODEL ##

GB_model = GradientBoostingClassifier(random_state=42)
GB_model.fit(X_train, y_train)

# Make predictions on validation set
y_val_pred = GB_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)

# Evaluate the validation set
print("Validation Accuracy:", val_accuracy)
print("Classification Report on Validation Set:\n", classification_report(y_val, y_val_pred, zero_division=1))
print("Confusion Matrix on Validation Set:\n", confusion_matrix(y_val, y_val_pred))

# Make predictions on test set
y_test_pred = LR_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Evaluate the test set
print("Test Accuracy:", test_accuracy)
print("Classification Report on Test Set:\n", classification_report(y_test, y_test_pred, zero_division=1))
print("Confusion Matrix on Test Set:\n", confusion_matrix(y_test, y_test_pred))

Validation Accuracy: 0.43333333333333335
Classification Report on Validation Set:
               precision    recall  f1-score   support

           1       0.50      0.60      0.55        15
           2       0.33      0.43      0.38         7
           3       0.33      0.12      0.18         8

    accuracy                           0.43        30
   macro avg       0.39      0.38      0.37        30
weighted avg       0.42      0.43      0.41        30

Confusion Matrix on Validation Set:
 [[9 5 1]
 [3 3 1]
 [6 1 1]]
Test Accuracy: 0.4
Classification Report on Test Set:
               precision    recall  f1-score   support

           1       1.00      0.33      0.50        18
           2       1.00      0.00      0.00         6
           3       0.25      1.00      0.40         6

    accuracy                           0.40        30
   macro avg       0.75      0.44      0.30        30
weighted avg       0.85      0.40      0.38        30

Confusion Matrix on Test Set:
 [[ 6

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [33]:
np.unique(y, return_counts=True)

(array([1, 2, 3]), array([84, 36, 28]))